In [1]:
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

### Loading data

In [3]:
s = hs.load("665.rpl").as_signal1D(0)

In [4]:
s.plot()

### Loading parameters

In [4]:
s.set_signal_type("EDS_SEM")

In [5]:
s.axes_manager[-1].name = 'E'
s.axes_manager['E'].units = 'keV'
s.axes_manager['E'].scale = 0.01
s.axes_manager['E'].offset = -0.1

In [6]:
s.add_elements(['Ge','Al','P','O'])

In [7]:
s.set_microscope_parameters(beam_energy=5.0)

In [8]:
s.add_lines(['O_Ka','Al_Ka','P_Ka','Ge_La'])

In [9]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       └── energy_resolution_MnKa = 130.0
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 5.0
├── General
│   ├── date = 
│   ├── original_filename = 665.rpl
│   ├── time = 
│   └── title = 
├── Sample
│   ├── elements = ['Al', 'Ge', 'O', 'P']
│   └── xray_lines = ['Al_Ka', 'Ge_La', 'O_Ka', 'P_Ka']
└── Signal
    ├── binned = True
    └── signal_type = EDS_SEM

In [10]:
s.crop_signal1D(-0.1, 2.5)

In [11]:
s.plot()

In [15]:
s.sum().plot(xray_lines=True)

In [11]:
s.change_dtype('float')

In [12]:
s.save('spectrumjuin.hdf5')

Overwrite 'spectrumjuin.hdf5' (y/n)?
y


In [14]:
### Loading data

In [13]:
sem_im = hs.load('Segmented_image_512.tif')

In [18]:
sem_im.plot()

In [14]:
s = hs.load('spectrumjuin.hdf5')

In [15]:
s.data.shape

(384, 512, 260)

In [16]:
sem_im.data.shape

(384, 512)

In [17]:
# Look for the pixel values in the SEM images
max_px_nbr = 3 # input the maximal number of different areas expected
px_values = np.zeros((max_px_nbr), dtype='int')

s_single_sum = hs.hyperspy.signal.hyperspy.signals.EDSSEMSpectrum( \
                    np.zeros([max_px_nbr,s.data.shape[2]], dtype='float'))

k_0=0
k_1=0
k_2=0
s_single_sum.data[:]=0

for i in range(0,sem_im.data.shape[0]):
    for j in range(0, sem_im.data.shape[1]):
        for k in range(0, px_values.size):
            if sem_im.isig[j,i]()[0]==0:
                s_single_sum.inav[0]=s_single_sum.inav[0]+s.inav[j,i]
                k_0 = k_0 +1
            elif sem_im.isig[j,i]()[0]==1:
                s_single_sum.inav[1]=s_single_sum.inav[1]+s.inav[j,i]
                k_1 = k_1 +1
            elif sem_im.isig[j,i]()[0]==2:
                s_single_sum.inav[2]=s_single_sum.inav[2]+s.inav[j,i]
                k_2 = k_2 +1
            else: print('pb')

In [18]:
s_single_sum2 = s_single_sum.deepcopy()

In [19]:
s_single_sum.add_elements(['Ge','Al','P','O'])

In [20]:
s_single_sum.axes_manager[1].offset = s.axes_manager[2].offset 
s_single_sum.axes_manager[1].scale = s.axes_manager[2].scale 
s_single_sum.axes_manager[1].units = s.axes_manager[2].units

In [21]:
s_single_sum.add_lines(['Al_Ka','P_Ka','Ge_La','O_Ka'])

In [22]:
s_single_sum.metadata.Acquisition_instrument = s.metadata.Acquisition_instrument
s_single_sum.metadata.Sample = s.metadata.Sample

In [23]:
s_single_sum.plot(xray_lines=True)

In [24]:
s_single_sum.save('Sum2JUIN.hdf5')

In [3]:
s_single_sum=hs.load('Sum2JUIN.hdf5')

In [4]:
s.plot()

NameError: name 's' is not defined

In [5]:
bw = s_single_sum.estimate_background_windows(line_width=[2.5, 2.0])
s_single_sum.plot(background_windows=bw)
intensities = s_single_sum.get_lines_intensity(background_windows=bw)

In [6]:
intensities

[<BaseSignal, title: X-ray line intensity of : Al_Ka at 1.49 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : Ge_La at 1.19 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : O_Ka at 0.52 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : P_Ka at 2.01 keV, dimensions: (3|)>]

In [7]:
# intensities are the intensities- 0: Al_Ka, 1: Ge_La, 2:O_Ka, 3:P_Ka 
# inav indicates which area to selec-t 0: inter-grain, 1:grain, 2:voids
# intensities are the X-rays intensities, they do not directly give the concentration
# To get the concentration the .quantification method should be used by needs the CL factors \
#       from the X-rays acquisition software

In [8]:
intensities[2]

<BaseSignal, title: X-ray line intensity of : O_Ka at 0.52 keV, dimensions: (3|)>

In [9]:
# Al/Ge inter-grain
intensities[0].inav[0].data[0]/intensities[1].inav[0].data[0]

0.16308582380282827

In [40]:
# Al/Ge grain
intensities[0].inav[1].data[0]/intensities[1].inav[1].data[0]

0.07284268393948509

In [41]:
# O/Ge inter-grain
intensities[2].inav[0].data[0]/intensities[1].inav[0].data[0]

2.9710211447353103

In [42]:
# O/Ge grain
intensities[2].inav[1].data[0]/intensities[1].inav[1].data[0]

2.164807022726413

In [43]:
# P/Ge inter-grain
intensities[3].inav[0].data[0]/intensities[1].inav[0].data[0]

1.0086961194350659

In [44]:
# P/Ge grain
intensities[3].inav[1].data[0]/intensities[1].inav[1].data[0]

0.6515988825024451

In [45]:
#O/Al inter-grain
intensities[2].inav[0].data[0]/intensities[0].inav[0].data[0]

18.217531576057112

In [46]:
#O/Al grain
intensities[2].inav[1].data[0]/intensities[0].inav[1].data[0]

29.718935459940653

In [47]:
#P/Al intergrain
intensities[3].inav[0].data[0]/intensities[0].inav[0].data[0]

6.185063152114223

In [48]:
#P/Al grain
intensities[3].inav[1].data[0]/intensities[0].inav[1].data[0]

8.945289317507418

In [49]:
#O/P intergrain
intensities[2].inav[0].data[0]/intensities[3].inav[0].data[0]

2.9454075290775106

In [50]:
#O/P grain
intensities[2].inav[1].data[0]/intensities[3].inav[1].data[0]

3.3223000850040427

In [62]:
###Ge grain/Ge inter grain
intensities[1].inav[1].data[0]/intensities[1].inav[0].data[0]

0.030729854302918084

In [63]:
#Al grain/Al inter grain
intensities[0].inav[1].data[0]/intensities[0].inav[0].data[0]

0.060111317254174394

In [64]:
#O grain/O inter grain
intensities[2].inav[1].data[0]/intensities[2].inav[0].data[0]

0.04388507273282118

In [65]:
#P grain/P inter grain
intensities[3].inav[1].data[0]/intensities[3].inav[0].data[0]

0.04170466793798192

In [10]:
s_single_sum.plot(xray_lines=True)

In [11]:
s_single_sum.inav[0].plot()
###spectra of intergrain region

In [12]:
s_single_sum.inav[1].plot()
###Spectra of the grain region

In [13]:
all_spectra= hs.plot.plot_spectra([s_single_sum.inav[0], s_single_sum.inav[1]])
### blue, yellow

In [14]:
s_single_sum.axes_manager[-1].name = 'E'
s_single_sum.axes_manager['E'].units = 'keV'

In [17]:
s_single_sum.add_elements(['Ge','Al','P','O'])
s_single_sum.add_lines(['Al_Ka','P_Ka','Ge_La','O_Ka'])
legendtext=['Inter-grain phase', 'Grains']
norma_all_spectra= hs.plot.plot_spectra([s_single_sum.inav[0], s_single_sum.inav[1]], legend=legendtext)
norma_all_spectra.plot(xray_lines=True, plot_result=True)
plt.draw()

In [49]:
all_spectra.figure.savefig("all_spectra_non_normalized.png")

In [15]:
### NORMALIZATION
for i in s_single_sum:
    i.data /= i.data.max()

In [16]:
norma_all_spectra= hs.plot.plot_spectra([s_single_sum.inav[0], s_single_sum.inav[1]])
plt.draw()
###blue intergrain, yellow grain

In [52]:
norma_all_spectra.figure.savefig("norm_spectra665.png")